In [1]:
!python -m pip install html2text

In [2]:
!python -m pip install mistletoe

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from contextlib import closing
from html2text import HTML2Text
from bs4 import BeautifulSoup
from pathlib import Path
from IPython.display import HTML as HTML_raw, display

import json
import sys
import re
import logging
import pandas as pd
import requests
import warcio
import mistletoe

# https://github.com/miyuchina/mistletoe

In [5]:
websites = pd.read_csv("./ccmain/ccmain_2021-04_Long2.txt", header = None, sep = '|') 

In [6]:
websites.head()

,0,1,2,3
0,NaN,url ...,warc_filename ...,NaN
1,NaN,https://globalannal.wordpress.com/tag/careers/...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
2,NaN,https://globalannal.wordpress.com/tag/jobs-num...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
3,NaN,https://globalannal.wordpress.com/tag/jobs-rep...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
4,NaN,https://globalshapersbucharest.wordpress.com/2...,crawl-data/CC-MAIN-2021-04/segments/1610703506...,NaN


In [7]:
websites.columns

Int64Index([0, 1, 2, 3], dtype='int64')

In [8]:
websites.drop([0, 3], axis=1, inplace=True)
websites.head()

,1,2
0,url ...,warc_filename ...
1,https://globalannal.wordpress.com/tag/careers/...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
2,https://globalannal.wordpress.com/tag/jobs-num...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
3,https://globalannal.wordpress.com/tag/jobs-rep...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
4,https://globalshapersbucharest.wordpress.com/2...,crawl-data/CC-MAIN-2021-04/segments/1610703506...


In [9]:
websites.columns = ['url_link', 'warc_file']
websites.head()

,url_link,warc_file
0,url ...,warc_filename ...
1,https://globalannal.wordpress.com/tag/careers/...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
2,https://globalannal.wordpress.com/tag/jobs-num...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
3,https://globalannal.wordpress.com/tag/jobs-rep...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
4,https://globalshapersbucharest.wordpress.com/2...,crawl-data/CC-MAIN-2021-04/segments/1610703506...


In [10]:
websites.url_link.value_counts()

https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      7
https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe                                                               

In [11]:
websites.url_link.value_counts().keys()

Index(['https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  ',
       'https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe                                                  

In [12]:
urlList = []
for i in websites.url_link.value_counts().keys():
    urlList.append(i.replace(' ',''))
    
print(urlList[:20])

['https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington', 'https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe', 'https://www.xn--itbaw-ova.de/jobs/151789/werkstudent-w-strich-m-strich-d-softwareentwicklung-java-in-stuttgart-und-karlsruhe', 'https://www.xn--itbaw-ova.de/jobs/152230/senior-software-engineer-w-strich-m-strich-d-sap-in-stuttgart-karlsruhe-und-reutlingen', 'https://www.vdk.de/bayern/pages/presse/vdk-zeitung/vdk-zeitung_archiv/80394/minijob_sichert_anspruch', 'https://www.workitdaily.com/should-quit-job-questions', 'https://www.xn--itbaw-ova.de/jobs/101860/senior-consultant-fuer-sap-banking-mit-sap-cml-strich-loans-management-m-strich-w-strich-d-bundesweit', 'https://www.xn--itbaw-ova.de/jobs/152501/consultant-w-strich-m-strich-d-sap-abap-in-stuttgart-karlsruhe-und-reutlingen', 'https://www.xn--itbaw-ova.de/jobs/152031/senior-consultant-w-

In [13]:
!python -m pip install cdx_toolkit

In [14]:
import cdx_toolkit

In [15]:
cdx = cdx_toolkit.CDXFetcher(source='cc')

In [16]:
cdx_indexes = requests.get('https://index.commoncrawl.org/collinfo.json').json()

In [17]:
pd.options.display.max_colwidth=150
pd.options.display.max_rows=6

In [18]:
pd.DataFrame(cdx_indexes)

,id,name,timegate,cdx-api
0,CC-MAIN-2021-04,January 2021 Index,https://index.commoncrawl.org/CC-MAIN-2021-04/,https://index.commoncrawl.org/CC-MAIN-2021-04-index
1,CC-MAIN-2020-50,November 2020 Index,https://index.commoncrawl.org/CC-MAIN-2020-50/,https://index.commoncrawl.org/CC-MAIN-2020-50-index
2,CC-MAIN-2020-45,October 2020 Index,https://index.commoncrawl.org/CC-MAIN-2020-45/,https://index.commoncrawl.org/CC-MAIN-2020-45-index
...,...,...,...,...
75,CC-MAIN-2012,Index of 2012 ARC files,https://index.commoncrawl.org/CC-MAIN-2012/,https://index.commoncrawl.org/CC-MAIN-2012-index
76,CC-MAIN-2009-2010,Index of 2009 - 2010 ARC files,https://index.commoncrawl.org/CC-MAIN-2009-2010/,https://index.commoncrawl.org/CC-MAIN-2009-2010-index
77,CC-MAIN-2008-2009,Index of 2008 - 2009 ARC files,https://index.commoncrawl.org/CC-MAIN-2008-2009/,https://index.commoncrawl.org/CC-MAIN-2008-2009-index


In [19]:
print(pd.DataFrame(cdx_indexes).head(20).to_markdown())

|    | id              | name                 | timegate                                       | cdx-api                                             |
|---:|:----------------|:---------------------|:-----------------------------------------------|:----------------------------------------------------|
|  0 | CC-MAIN-2021-04 | January 2021 Index   | https://index.commoncrawl.org/CC-MAIN-2021-04/ | https://index.commoncrawl.org/CC-MAIN-2021-04-index |
|  1 | CC-MAIN-2020-50 | November 2020 Index  | https://index.commoncrawl.org/CC-MAIN-2020-50/ | https://index.commoncrawl.org/CC-MAIN-2020-50-index |
|  2 | CC-MAIN-2020-45 | October 2020 Index   | https://index.commoncrawl.org/CC-MAIN-2020-45/ | https://index.commoncrawl.org/CC-MAIN-2020-45-index |
|  3 | CC-MAIN-2020-40 | September 2020 Index | https://index.commoncrawl.org/CC-MAIN-2020-40/ | https://index.commoncrawl.org/CC-MAIN-2020-40-index |
|  4 | CC-MAIN-2020-34 | August 2020 Index    | https://index.commoncrawl.org/CC-MAIN-2020-34/

In [20]:
print(pd.DataFrame(cdx_indexes).tail(20).to_markdown())

|    | id                | name                           | timegate                                         | cdx-api                                               |
|---:|:------------------|:-------------------------------|:-------------------------------------------------|:------------------------------------------------------|
| 58 | CC-MAIN-2015-32   | July 2015 Index                | https://index.commoncrawl.org/CC-MAIN-2015-32/   | https://index.commoncrawl.org/CC-MAIN-2015-32-index   |
| 59 | CC-MAIN-2015-27   | June 2015 Index                | https://index.commoncrawl.org/CC-MAIN-2015-27/   | https://index.commoncrawl.org/CC-MAIN-2015-27-index   |
| 60 | CC-MAIN-2015-22   | May 2015 Index                 | https://index.commoncrawl.org/CC-MAIN-2015-22/   | https://index.commoncrawl.org/CC-MAIN-2015-22-index   |
| 61 | CC-MAIN-2015-18   | April 2015 Index               | https://index.commoncrawl.org/CC-MAIN-2015-18/   | https://index.commoncrawl.org/CC-MAIN-2015-18-index   

In [21]:
api_url = cdx_indexes[0]['cdx-api']
api_url

'https://index.commoncrawl.org/CC-MAIN-2021-04-index'

In [22]:
f = open(r"ccmain_2021jan_jobURL.txt","a+")
for url_link in urlList:
    r = requests.get(api_url,
                 params = {
                     'url': url_link,
                     'output': 'json',
                     'filter': ['=status:200', '=mime-detected:text/html']
                 })
    records = [json.loads(line) for line in r.text.split('\n') if line]
    df = pd.DataFrame(records)
    print(url_link,' ',df.shape[0])
    L = [str(url_link),' ',str(df.shape[0]), '\n']
    f.writelines(L) 
f.close()

https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington   7
https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe   5
https://www.xn--itbaw-ova.de/jobs/151789/werkstudent-w-strich-m-strich-d-softwareentwicklung-java-in-stuttgart-und-karlsruhe   4
https://www.xn--itbaw-ova.de/jobs/152230/senior-software-engineer-w-strich-m-strich-d-sap-in-stuttgart-karlsruhe-und-reutlingen   4
https://www.vdk.de/bayern/pages/presse/vdk-zeitung/vdk-zeitung_archiv/80394/minijob_sichert_anspruch   1
https://www.workitdaily.com/should-quit-job-questions   3
https://www.xn--itbaw-ova.de/jobs/101860/senior-consultant-fuer-sap-banking-mit-sap-cml-strich-loans-management-m-strich-w-strich-d-bundesweit   3
https://www.xn--itbaw-ova.de/jobs/152501/consultant-w-strich-m-strich-d-sap-abap-in-stuttgart-karlsruhe-und-reutlingen   3
https://www.xn--itbaw-ova.de/jobs/152031/senior-consult

https://www.workselection.com/job/manager-people-culture-m-w-d-60-triengen-6606/   2
https://www.workselection.com/job/dachdecker-region-baselland-m-w-d-basel-6576/   2
https://internationalsecuritydiscipulus.wordpress.com/2013/09/19/so-you-want-a-job-in-policy/   2
https://www.workselection.com/job/process-engineer-pfmea-m-w-d-rotkreuz-6695/   2
https://www.xn--itbaw-ova.de/jobs/101853/softwareentwickler-abap-fuer-sap-banking-m-strich-w-strich-d-bundesweit   2
https://vancouverwritingjobs.wordpress.com/tag/vancouver-media-jobs/   2
https://www.wkr-anwalt.de/ueber-wkr/job-karriere/rechtsanwaelte/   2
https://www.workselection.com/job/admin-assistant-procurement-m-f-d-rotkreuz-6665/   2
http://www.vaybee.de/nachrichten/job-karriere/   2
https://resources.workable.com/hr-onboarding-specialist-job-description   2
https://www.workselection.com/job/dachdecker-region-aargau-m-w-d-baden-6613/   2
https://lovetravelandcoffeetravel.wordpress.com/category/travel-jobs/   2
https://www.wilabonn.de

https://violetdaniels997.wordpress.com/tag/career/   2
https://www.workselection.com/job/quality-manager-operations-m-f-d-rotkreuz-6586/   2
https://www.workselection.com/job/clinical-research-associate-m-f-d-rotkreuz-6542/   2
https://www.workinfinland.com/jobs-in-finland/   2
https://jacquiwucom.wordpress.com/tag/career/   2
https://help.workable.com/hc/en-us/articles/360042905674-How-do-I-post-a-job-in-multiple-languages-   2
https://www.workselection.com/job/bauleiter-innenausbau-m-w-d-duebendorf-6679/   2
https://www.workselection.com/job/landschaftsg-rtner-region-baselland-m-w-d-basel-6566/   2
https://www.workselection.com/job/project-leader-consumables-80-100-m-f-d-rotkreuz-6582/   2
https://www.workselection.com/job/zimmermann-region-basel-m-w-d-basel-6659/   2
https://www.workselection.com/job/zimmermann-m-w-d-basel-6681/   2
https://workingthoughts.com/2010/04/10/is-the-job-market-ready-for-good-news/   1
https://www.workselection.com/job/kundeng-rtner-region-baselland-m-w-d

https://www.workitdaily.com/fix-job-gap-work-history   2
https://www.xn--itbaw-ova.de/jobs/151932/competence-center-leitung-w-strich-m-strich-d-java-in-stuttgart   2
https://www.workselection.com/job/erfahrener-sachbearbeiter-leistungsmanagement-m-w-d-basel-6664/   2
https://resources.workable.com/senior-system-administrator-job-description   2
https://www.xn--itbaw-ova.de/jobs/122820/linux-systemadministrator-strich-system-engineer-m-strich-w-strich-x-im-home-office   2
https://www.workinsports.com/wisquickregseojob-fr.asp?idx=502384   2
https://www.vogt-klimatechnik.de/jobs-und-karriere/handwerksmeister-als-montageleiter-m-w-vollzeit/   2
https://www.workitdaily.com/biggest-secret-obstacle-career   2
https://resources.workable.com/web-designer-job-description   2
https://www.workselection.com/job/sanit-rmonteur-region-solothurn-m-w-d-basel-6562/   2
https://www.workselection.com/job/dachdecker-region-z-rich-m-w-d-zuerich-6537/   2
https://lucas2012infos.wordpress.com/2014/08/16/arab-

https://www.workitdaily.com/crisis-confidence-job-search   2
https://www.workselection.com/job/bauspengler-region-solothurn-m-w-d-basel-6577/   2
https://www.workselection.com/job/parkettleger-region-z-rich-m-w-d-zuerich-6545/   2
https://www.workselection.com/job/bauspengler-region-aargau-m-w-d-baden-6616/   2
http://worksource1.com/job-seeker/disability-resource-coordination   2
https://wirsuchenmenschen.de/azubi-job-matching/results/   2
https://www.workitdaily.com/annual-career-checkup   2
https://www.workselection.com/job/technischer-eink-ufer-m-w-d-f-r-projektarbeitaarau-6596/   2
https://vdi-schwarzwald.de/regio-career/   3
https://www.workitdaily.com/setting-career-goals   2
https://komigenuva.wordpress.com/2012/06/29/spelmissbruk-den-jobblosa-tillvaxten-och-kapitalackumulationen-bailas-ut-en-gang-till/   2
https://www.workselection.com/job/rohrschlosser-in-der-region-baselland-m-w-d-muttenz-6591/   2
https://www.workselection.com/job/sanit-rmonteur-region-aargau-m-w-d-kaiserau

...

https://www.workventure.com/jobs/for/%E0%B9%80%E0%B8%88%E0%B9%89%E0%B8%B2%E0%B8%AB%E0%B8%99%E0%B9%89%E0%B8%B2%E0%B8%97%E0%B8%B5%E0%B9%88%E0%B8%84%E0%B8%A7%E0%B8%9A%E0%B8%84%E0%B8%B8%E0%B8%A1%E0%B9%80%E0%B8%84%E0%B8%A3%E0%B8%B7%E0%B9%88%E0%B8%AD%E0%B8%87%E0%B8%84%E0%B8%AD%E0%B8%A1%E0%B8%9E%E0%B8%B4%E0%B8%A7%E0%B9%80%E0%B8%95%E0%B8%AD%E0%B8%A3%E0%B9%8C   1
https://www.worknetstaffing.com/jobs/production-sewing-worker/   1
https://www.workinsports.com/wisquickregseojob-fr.asp?idx=502028   1
http://www.xn--jobs-fr-bochum-lsb.de/job/microsoft-azure-m365-consultant.492897315.html   1
https://www.workopolis.com/jobsearch/viewjob/_BleduUs8xGHlP5lBWW6YIxT0V4DHhsdXoThHjG7zPifEJhvieruBA?ak=&l=saint+jean+sur+richelieu+quebec+province   1
http://www.wirsuchenwiedereinsteiger.de/jobs/ausbildung-zur-pflegefachperson-m-w-d-klinik-koenigshof/   2
https://workinstartups.com/job-board/job/100002/e-commerce-associate-london-living-wage-perfect-for-masters-students-at-megamac/   1
https://mathspig.wordpres

https://www.world-working.com/view-one-job/17601/   1
https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?tjt=Facilities&q=occda&where=Washington&rad=30&sort=rv.di.dt&amp;rad_units=miles&amp;pp=25&amp;vw=b&amp;setype=2&amp;tm=0&amp;jbf47538=1&brd=3&tm=-1   1
https://thefamilyy.workbrightats.com/jobs/487943.html   1
https://wirsuchenmenschen.de/jobs/mitarbeiter-fuer-die-menueasistenz-m-w-d-geringfuegige-beschaeftigung/   1
https://www.workana.com/es/job/diseno-y-asesoria-en-produccion-de-reconocimiento   1
http://www.wirsuchenwiedereinsteiger.de/jobs/mitarbeiter-m-w-d-in-der-hauswirtschaft-johana-etiene-krankenhaus0/   3
https://gunnhelene.wordpress.com/2015/07/01/jeg-vil-ha-ny-jobb/   1
https://wir-suchen-bachelor-pflegefachfrauen.de/jobs/physiotherapeut-m-w-d-ambulantes-team-am-kreiskrankenhaus-grevenbroich/   1
https://python.works-hub.com/jobs/elixir-software-engineer-mid-senior-in-barcelona-spain-e2d0e   1
https://workinpenang.com/job-tag/happy/   1
https://kwoted.wordpress.

https://www.xn--gamsjger-puchheim-uqb.de/job-finden   1
https://www.workana.com/job/insight/textos-sobre-marketing-digital-e-ecommerce-4-mes?ref=project_view   1
https://www.workthewilds.com/jobs/listing/ZQ%3D%3D   1
https://seeker.worksourcewa.com/jobsearch/PowerSearch.aspx?tm=30&q=occda&rad=20&sort=dt.rv.di&amp;rad_units=miles&amp;pp=25&amp;vw=b&amp;setype=2&amp;rad=30&brd=3&brd=1   1
https://www.workana.com/pt/job/artigos-para-o-site-namoro-dos-sonhos-2   1
https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?tm=30&q=occda&sort=rv.di.dt&amp;amp;rad_units=miles&amp;amp;pp=25&amp;amp;vw=b&amp;amp;setype=2&amp;amp;rad=10&amp;rad=30&brd=3%2C1&jbf67816=88993   1
https://www.workana.com/pt/job/colab-site-igual-ao-touts-com-br   1
https://resources.workable.com/compliance-officer-job-description   1
https://workinstartups.com/job-board/job/100181/sales-development-representative-at-infloai/   1
https://seeker.worksourcewa.com/jobsearch/PowerSearch.aspx?jt=3&tm=30&q=occda&where=Washin

https://www.worklooker.com/la-baton-rouge-parish-jobs.php   1
https://workneet.com/experience-note/part-time-job-experience/   1
https://www.workopolis.com/jobsearch/viewjob/64psLq44UJpYRwsc13K7lEZV88QeStuFlLpHdwrks_qxzLtFkTDTcg?ak=&l=Carleton+Place%2C+ON   1
https://wirsuchenmenschen.de/jobs/case-manager-m-w-d0/   1
http://www.wirsuchenheilerziehungspfleger.de/jobs/sozialarbeiter-m-w-d-fuer-den-kliniksozialdienst/   2
https://www.workana.com/pt/job/revitalizacao-de-site-provedor-de-internet-wordpress   1
https://www.workopolis.com/jobsearch/viewjob/Ewpuy2Ng77xs9lOg0fQd-TbJFN8fiv0Qo0TAy0woQ4jTQUJ13P6yrg   1
https://workingnotworking.com/search/jobs?search_id=24432198   1
https://kwoted.wordpress.com/2016/12/30/the-ultimate-inside-job-you/?like_comment=4532&_wpnonce=75e6ce629f   1
https://kdepepo.wordpress.com/2013/09/13/about-a-qml-junior-job/   1
https://www.wiwi-treff.de/Studentenjob-and-Finanzen/Studienfinanzierung/Studentenwerke-Stipendien-und-Kredite-koennen-das-BAfoeG-nur-margina

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

cdxt --cc --from 202101 --to 202102 --filter '=status:200' iter 'https://www.workana.com' --fields url | wc -l

> 55

cdxt --cc --from 202101 --to 202102 --filter '=status:200' iter 'https://www.workinsports.com' --fields url | wc -l

> 1

cdxt --cc --from 202101 --to 202102 --filter '=status:200' iter 'https://seeker.worksourcewa.com' --fields url | wc -l

> 1

cdxt --cc --from 202101 --to 202102 --filter '=status:200' iter 'https://kwoted.wordpress.com' --fields url | wc -l

> 1

cdxt --cc --from 202101 --to 202102 --filter '=status:200' iter 'https://www.xn--jobbrse-stellenangebote-blc.de' --fields url | wc -l

> 5

cdxt --cc --from 202101 --to 202102 --filter '=status:200' iter 'https://www.workitdaily.com' --fields url | wc -l

> 1

cdxt --cc --from 202101 --to 202102 --filter '=status:200' iter 'https://workinhostels.com' --fields url | wc -l

> 1

cdxt --cc --from 202101 --to 202102 --filter '=status:200' iter 'https://workshift-sol.com' --fields url | wc -l

> 1